In [1]:
import os
os.chdir("../")

In [3]:
%pwd

'C:\\Users\\Administrator\\Desktop\\SolarHotWaterSolution\\F1-Backend'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')


In [11]:
#Spliat the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [13]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))


Length of Text Chunks 21


In [15]:
from langchain_openai  import OpenAIEmbeddings

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [21]:
print(PINECONE_API_KEY)


pcsk_5MJnqE_MDFbtp5aVBAmPyPN2N8HzFdoAMQWUiRrbzei6GYi6CjRcQkiimBGw2G6sRqXmhw


In [23]:
def download_openai_embeddings():
    """Downloads and returns OpenAI embeddings."""
    embeddings = OpenAIEmbeddings()
    return embeddings


embeddings = download_openai_embeddings()

query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 1536


In [29]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "davisandshirliff"


pc.create_index(
    name=index_name,
    dimension=1536, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [30]:
index_name = "davisandshirliff"

In [31]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [35]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [37]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [39]:
retrieved_docs = retriever.invoke("Ultrasun UF X300I Solar Water Heating System?")

In [41]:
retrieved_docs

[Document(id='e92f099c-428d-4828-9435-bfb1d9432c1c', metadata={'page': 0.0, 'source': 'Data\\Ultrasun UFX.pdf'}, page_content='• Galvanised mounting frame for both flat or inclined roof installations that ensures high durability in all weather conditions \nUltrasun UFX Flatplate  solar systems are available in various tank sizes and collector configurations to suit domestic and small- scale \ninstallation applications. They are efficient and robust products guaranteed for five years to demonstrate the high material specification. \nDesigned for many years of trouble free operations, they are the ideal solution for all solar water heating applications.  \nSPECIFICA TIONS\nWater Quality: Water outside the following limits should be appropriately pre-treated, \nDirect : Clarity : clear, TDS<600mg/l, Hardness <200mg/l CaCO Saturation Index: >0.8<1.0  3, \nIndirect:  Clarity:  clear, TDS: <1500mg/l,  Hardness:  <400mg/l CaCo , Saturation Index: >0.8<1.0 3\nMax. Operating Pressure: 6 Bar for

In [47]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [48]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [51]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [53]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [55]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [59]:
response = rag_chain.invoke({"input": "Tell me more about Ultrasun UF X300I Solar Water Heating System. "})
print(response["answer"])



The Ultrasun UFX300I Solar Water Heating System is a high specification thermosyphon type hot water heater designed for long life operation in extreme conditions. It is available in both direct and indirect options, with the direct system heating the water directly through the collector and the indirect system separating the collector flow water from the through flow water. This system is suitable for both domestic and small-scale installation applications and is guaranteed for five years. It is designed for trouble-free operation and is efficient and robust.
